In [0]:
with cte_data (y, b) as
(
select monotonically_increasing_id() as y
     , transform(split(value, ''), t -> (t = '@')::tinyint)
--from read_files('/Volumes/waldar/fabien/aoc2025/aoc2025_04_input.txt', format => 'text', wholeText => false) as t (value)
  from values ('..@@.@@@@.')
            , ('@@@.@.@.@@')
            , ('@@@@@.@.@@')
            , ('@.@@@@..@.')
            , ('@@.@@@@.@@')
            , ('.@@@@@@@.@')
            , ('.@.@.@.@@@')
            , ('@.@@@.@@@@')
            , ('.@@@@@@@@.')
            , ('@.@.@@@.@.') as t (value)
)
  ,  cte_grid (g) as
(
select array_sort(array_agg(struct(y, b))).b
  from cte_data
)
select aggregate( transform( g
                           , (u, y) -> array_size( filter( transform( u
                                                                   , (t, x) -> case t
                                                                                 when 0
                                                                                 then 0
                                                                                 else coalesce(g[y-1][x-1], 0) + coalesce(g[y-1][x], 0) + coalesce(g[y-1][x+1], 0)
                                                                                    + coalesce(g[ y ][x-1], 0) +            1           + coalesce(g[ y ][x+1], 0)
                                                                                    + coalesce(g[y+1][x-1], 0) + coalesce(g[y+1][x], 0) + coalesce(g[y+1][x+1], 0)
                                                                               end )
                                                         , f -> f between 1 and 4 ) ) )
               , 0, (acc, n) -> acc + n
               ) as part1
     , aggregate( sequence(0,100)
                , struct(0 as r, -1 as pr, g as arr)
                , (agg, a) -> case agg.r
                                when agg.pr
                                then agg
                                else aggregate( array()::array<int>
                                              , transform( agg.arr
                                                   , (u, y) -> transform( u
                                                                   , (t, x) -> case t
                                                                                 when 0
                                                                                 then 0
                                                                                 else coalesce(agg.arr[y-1][x-1], 0)
                                                                                    + coalesce(agg.arr[y-1][ x ], 0)
                                                                                    + coalesce(agg.arr[y-1][x+1], 0)
                                                                                    + coalesce(agg.arr[ y ][x-1], 0)
                                                                                    + 1
                                                                                    + coalesce(agg.arr[ y ][x+1], 0)
                                                                                    + coalesce(agg.arr[y+1][x-1], 0)
                                                                                    + coalesce(agg.arr[y+1][ x ], 0)
                                                                                    + coalesce(agg.arr[y+1][x+1], 0)
                                                                               end ) )
                                              , (azz, z) -> null
                                              , azz -> struct( agg.r + aggregate(transform( azz, u -> array_size(filter(u, f -> f between 1 and 4 ))), 0, (acc, c) -> acc + c) as r
                                                             , agg.r as pr
                                                             , transform(azz, u -> transform(u, t -> (t > 4)::tinyint)) as arr
                                                             )
                                              )
                              end
                , agg -> agg.r
                ) as part2
  from cte_grid;

